In [1]:
import torch

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cpu')

In [5]:
torch.manual_seed(42)

In [6]:
m = torch.rand(2, 3); m

tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009]])

In [7]:
type(m)

torch.Tensor

In [8]:
m = torch.rand((2, 3)); m

tensor([[0.2566, 0.7936, 0.9408],
        [0.1332, 0.9346, 0.5936]])

In [5]:
import numpy as np

In [52]:
n = np.random.rand(2, 3); n

array([[0.26322173, 0.84616642, 0.82173497],
       [0.36775626, 0.16292076, 0.04979593]])

In [19]:
m.reshape(1,6)

tensor([[0.1053, 0.2695, 0.3588, 0.1994, 0.5472, 0.0062]])

In [21]:
torch.ones(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [23]:
torch.ones((2, 3))

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [25]:
torch.equal(torch.ones(2, 3), torch.ones((2, 3)))

True

In [26]:
torch.ones(2, 3, dtype=torch.int)

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int32)

In [28]:
torch.Tensor([1, 2, 3])

tensor([1., 2., 3.])

In [30]:
torch.tensor([1, 2, 3])

tensor([1, 2, 3])

In [31]:
torch.Tensor([1, 2, 3]).dtype

torch.float32

In [32]:
torch.tensor([1, 2, 3]).dtype

torch.int64

In [35]:
torch.Tensor([1, 2, 3], dtype=torch.float16)

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [39]:
torch.tensor([1, 2, 3], dtype=torch.float16, device=device)

tensor([1., 2., 3.], dtype=torch.float16)

In [40]:
torch.tensor([1, 2, 3]).shape

torch.Size([3])

In [41]:
torch.ones((2, 3)).shape

torch.Size([2, 3])

In [42]:
torch.ones((2, 3)).ndim

2

In [43]:
m[1, 1]

tensor(0.5472)

In [44]:
m[1, 1].item()

0.5471915602684021

In [45]:
m.shape

torch.Size([2, 3])

In [46]:
m.reshape(-1, 6)

tensor([[0.1053, 0.2695, 0.3588, 0.1994, 0.5472, 0.0062]])

In [47]:
m.reshape(-1, 1)

tensor([[0.1053],
        [0.2695],
        [0.3588],
        [0.1994],
        [0.5472],
        [0.0062]])

In [48]:
m.numpy()

array([[0.10531491, 0.26949483, 0.35881263],
       [0.19936377, 0.54719156, 0.00616044]], dtype=float32)

In [49]:
m + 1

tensor([[1.1053, 1.2695, 1.3588],
        [1.1994, 1.5472, 1.0062]])

In [50]:
m.add_(1) # inplace

tensor([[1.1053, 1.2695, 1.3588],
        [1.1994, 1.5472, 1.0062]])

In [53]:
torch.from_numpy(n)

tensor([[0.2632, 0.8462, 0.8217],
        [0.3678, 0.1629, 0.0498]], dtype=torch.float64)

In [ ]:
torch.tensor()